In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
dataset=pd.read_csv("/kaggle/input/titanic/train.csv")
dataset.head()
dataset2=pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
x=dataset.iloc[:,2:].values
ID=dataset2.iloc[:,0].values
x_test=dataset2.iloc[:,1:].values

y=dataset.iloc[:,1].values
y=y.reshape(len(y),1)


x=np.delete(x,1,1)
x_test=np.delete(x_test,1,1)
print(x[0:3])
x=np.delete(x,(5,7),1)
x_test=np.delete(x_test,(5,7),1)
print(" ")
print(x[0:5])
x[:,3]=x[:,3]+x[:,4]
x_test[:,3]+=x_test[:,3]+x_test[:,4]

x=np.delete(x,4,1)
x_test=np.delete(x_test,4,1)
print(" ")
print(x[0:5])

[[3 'male' 22.0 1 0 'A/5 21171' 7.25 nan 'S']
 [1 'female' 38.0 1 0 'PC 17599' 71.2833 'C85' 'C']
 [3 'female' 26.0 0 0 'STON/O2. 3101282' 7.925 nan 'S']]
 
[[3 'male' 22.0 1 0 7.25 'S']
 [1 'female' 38.0 1 0 71.2833 'C']
 [3 'female' 26.0 0 0 7.925 'S']
 [1 'female' 35.0 1 0 53.1 'S']
 [3 'male' 35.0 0 0 8.05 'S']]
 
[[3 'male' 22.0 1 7.25 'S']
 [1 'female' 38.0 1 71.2833 'C']
 [3 'female' 26.0 0 7.925 'S']
 [1 'female' 35.0 1 53.1 'S']
 [3 'male' 35.0 0 8.05 'S']]


In [4]:
import tensorflow as tf

In [5]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
le2=LabelEncoder()
x[:,1]=le1.fit_transform(x[:,1])
x_test[:,1]=le1.transform(x_test[:,1])

#x[:,0]=le2.fit_transform(x[:,0])
#x_test[:,0]=le2.transform(x_test[:,0])
print(x[0:5])

[[3 1 22.0 1 7.25 'S']
 [1 0 38.0 1 71.2833 'C']
 [3 0 26.0 0 7.925 'S']
 [1 0 35.0 1 53.1 'S']
 [3 1 35.0 0 8.05 'S']]


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct1=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[5])],remainder='passthrough')
x=np.array(ct1.fit_transform(x))                     
x_test=np.array(ct1.transform(x_test))

ct2=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
x=np.array(ct2.fit_transform(x))                     
x_test=np.array(ct2.transform(x_test))

In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(x)
x = imputer.transform(x)
x_test=imputer.transform(x_test)

In [8]:
x_train, y_train=x,y

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [10]:
'''from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)'''

'''from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)'''

'''from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski', p = 2)'''

'''from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)'''

from xgboost import XGBClassifier
classifier= XGBClassifier()

classifier.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [11]:
y_pred=classifier.predict(x_test)

In [12]:
output = pd.DataFrame({'PassengerId':ID, 'Survived':y_pred})
output.to_csv('submission.csv',index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
